In [ ]:
import os

import openai
os.environ['OPENAI_API_KEY'] = "sk-48characterstofakeanopenaikey48charactersopenai0"
os.environ['OPENAI_API_BASE'] = "http://10.0.0.222:30307/v1"
openai.api_key = "sk-48characterstofakeanopenaikey48charactersopenai0"
openai.api_base = "http://10.0.0.222:30307/v1"

model = "Writer/camel-5b-hf"
#model = "mosaicml/mpt-7b-instruct"
#model = "mosaicml/mpt-30b-instruct"

INDEX_NAME = f"{model.replace('/', '-')}-default-no-coref"
#INDEX_NAME = f"{model.replace('/', '-')}-default-coref"
persist_path = f"storage/{INDEX_NAME}"

In [ ]:
import logging
import sys

#kron extensions to llama_index to support openai compatible api
sys.path.append('../llama-index/')

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Using Knowledge Graph

In [ ]:
### Query Knowledge Graph

In [ ]:

from llama_index import StorageContext
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.graph_stores import SimpleGraphStore 
from llama_index import load_index_from_storage 
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser

import tiktoken

#extensions to llama_index to support openai compatible endpoints, e.g. llama-api
from kron.llm_predictor.KronOpenAILLM import KronOpenAI
from kron.llm_predictor.KronLLMPredictor import KronLLMPredictor
#from kron.indices.knowledge_graph.KronKnowledgeGraphIndex import KronKnowledgeGraphIndex 
#from kron.prompts.kg_prompts import KRON_KG_TRIPLET_EXTRACT_PROMPT

In [ ]:
#writer/camel uses endoftext 
from llama_index.utils import globals_helper
enc = tiktoken.get_encoding("gpt2")
tokenizer = lambda text: enc.encode(text, allowed_special={"<|endoftext|>"})
globals_helper._tokenizer = tokenizer

In [ ]:
# define LLM
llm=KronOpenAI(temperature=0.01, model=model)
llm.max_tokens = 1536 
llm_predictor = KronLLMPredictor(llm)
print(llm_predictor.metadata)

In [ ]:
# define TextSplitter
text_splitter = SentenceSplitter(chunk_size=192, chunk_overlap=48, paragraph_separator='\n')
#define NodeParser
node_parser = SimpleNodeParser(text_splitter=text_splitter)
#define ServiceContext
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, node_parser=node_parser)

In [ ]:
print(f'Loading index from {persist_path}')
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=persist_path)
# load index
index = load_index_from_storage(storage_context=storage_context, 
                                    service_context=service_context, 
                                    max_triplets_per_chunk=2,
                                    show_progress = True)

In [ ]:
#writer/camel does not understand the refine prompt
RESPONSE_MODE = 'accumulate'
query_engine = index.as_query_engine(response_mode = RESPONSE_MODE)

In [ ]:
response = query_engine.query("What is SPGC?")
print(response)

In [ ]:
response = query_engine.query("What is AD recognition?")
print(response)

In [ ]:
response = query_engine.query("What is the relationship between AD recognition and SPGC?")
print(response)
